In [5]:
from typing import Dict, Tuple, List, Any
from pathlib import Path
import pandas as pd
from backtesting import Strategy, Backtest
from kedro.framework.session import KedroSession
from kedro.framework.startup import bootstrap_project

In [2]:
metadata = bootstrap_project(Path.cwd().parent.parent)
with KedroSession.create(metadata.package_name,
        project_path=metadata.project_path,
        env="base",
    ) as session: 
    context = session.load_context()

/usr/local/lib/python3.8/site-packages/kedro/config/config.py:296: UserWarning: Duplicate environment detected! Skipping re-loading from configuration path: /backtest-example/backtest-example/conf/base
  warn(


2022-01-01 11:51:19,164 - kedro.framework.session.store - INFO - `save()` not implemented for `BaseSessionStore`. Skipping the step.


In [3]:
class LgbmModel(Strategy):

    def init(self):
        pass

    def next(self):
        """
        next() function is executed per bar.
        We can get latest value access to index of [-1].
        """
        # exit
        if self.data.buy_cost[-1]:
            self.position.close()


        if self.data.sell_cost[-1]:
            self.position.close()

        # entry
        if self.data.buy_entry[-1] and self.data.buy_cost[-1]:
            self.buy()

        if self.data.sell_entry[-1] and self.data.sell_cost[-1]:
            self.sell()

In [13]:
def supertrend_test(df: pd.DataFrame):
    """
    ref: https://gist.github.com/shinseitaro/be1d1ea012b1dc2657569632eb1e9069#file-crypto-advent-calender-2021-ipynb
    """
    bt = Backtest(df, LgbmModel, trade_on_close=True, cash=100000, commission=0.001)
    stats = bt.run()

    return (bt, stats)

In [10]:
predicted = context.catalog.load("include_y_hat")
predicted["buy_entry"] = predicted["y_hat_buy"].values > 0
predicted["sell_entry"] = predicted["y_hat_sell"].values > 0
bt, stats = supertrend_test(predicted)

2022-01-01 12:09:20,140 - kedro.io.data_catalog - INFO - Loading data from `include_y_hat` (PickleDataSet)...


/usr/local/lib/python3.8/site-packages/kedro/config/config.py:296: UserWarning: Duplicate environment detected! Skipping re-loading from configuration path: /backtest-example/backtest-example/conf/base
  warn(
/usr/local/lib/python3.8/site-packages/kedro/framework/context/context.py:488: UserWarning: Credentials not found in your Kedro project config.
No files found in ['/backtest-example/backtest-example/conf/base'] matching the glob pattern(s): ['credentials*', 'credentials*/**', '**/credentials*']
  warn(f"Credentials not found in your Kedro project config.\n{str(exc)}")


In [11]:
stats

Start                     2021-12-26 14:00:00
End                       2021-12-31 11:10:00
Duration                      4 days 21:10:00
Exposure Time [%]                    52.84495
Equity Final [$]                    51400.216
Equity Peak [$]                      100000.0
Return [%]                         -48.599784
Buy & Hold Return [%]               -3.678327
Return (Ann.) [%]                      -100.0
Volatility (Ann.) [%]                     0.0
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -48.599784
Avg. Drawdown [%]                  -48.599784
Max. Drawdown Duration        4 days 01:30:00
Avg. Drawdown Duration        4 days 01:30:00
# Trades                                  411
Win Rate [%]                         4.622871
Best Trade [%]                       0.904001
Worst Trade [%]                     -1.576236
Avg. Trade [%]                    

In [12]:
bt.plot()

Row(id='1408', ...)